<a href="https://colab.research.google.com/github/uoacapstonegroup6/CapstoneUOATeam6/blob/main/CapstoneProject_MedVit_FractureMNIST3d_FinalCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the MedMNIST library, a large-scale benchmark dataset for medical image classification tasks

!pip install medmnist

In [ ]:
%cd '/home/exouser/Documents/RRR_MedVit'

/home/exouser/Documents/RRR_MedVit


/home/exouser/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
%pwd

'/home/exouser/Documents/RRR_MedVit'

# Import Library

!pip install torchsummary

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary

from tqdm import tqdm
import numpy as np

import torch.utils.data as data
import torchvision.transforms as transforms
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

import medmnist
from medmnist import INFO, Evaluator

from PIL import Image

# Download Data

In [ ]:
# data_flag = 'vesselmnist3d'
# data_flag='synapsemnist3d'
data_flag = 'fracturemnist3d'

In [ ]:

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])
train_dataset = DataClass(split='train', download=True,size=64)
val_dataset = DataClass(split='val', download=True,size=64)
test_dataset = DataClass(split='test', download=True,size=64)



Using downloaded and verified file: /home/exouser/.medmnist/fracturemnist3d_64.npz
Using downloaded and verified file: /home/exouser/.medmnist/fracturemnist3d_64.npz
Using downloaded and verified file: /home/exouser/.medmnist/fracturemnist3d_64.npz


In [ ]:
# Create directories for saving 2D frames
train_dir = f'./{data_flag}/train'
valid_dir = f'./{data_flag}/valid'
test_dir = f'./{data_flag}/test'

In [ ]:

for dir in [train_dir, valid_dir, test_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)

# Extract 2D slices from 3D data and save to local drive
def extract_and_save_2d_slices(dataset, dir):
    for idx, img in enumerate(dataset.imgs):
        label = dataset.labels[idx]
        for i in range(img.shape[0]):  # Extract each of the 64 frames
            slice_2d = img[i, :, :]
            slice_2d_img = Image.fromarray(slice_2d)
            filename = f"{idx}_{i}.png"
            filepath = os.path.join(dir, str(label), filename)
            if not os.path.exists(os.path.dirname(filepath)):
                os.makedirs(os.path.dirname(filepath))
            slice_2d_img.save(filepath)

extract_and_save_2d_slices(train_dataset, train_dir)
extract_and_save_2d_slices(val_dataset, valid_dir)
extract_and_save_2d_slices(test_dataset, test_dir)

# Data Preprocessing

In [ ]:
print(test_dataset)

Dataset FractureMNIST3D of size 64 (fracturemnist3d_64)
    Number of datapoints: 240
    Root location: /home/exouser/.medmnist
    Split: test
    Task: multi-class
    Number of channels: 1
    Meaning of labels: {'0': 'buckle rib fracture', '1': 'nondisplaced rib fracture', '2': 'displaced rib fracture'}
    Number of samples: {'train': 1027, 'val': 103, 'test': 240}
    Description: The FractureMNIST3D is based on the RibFrac Dataset, containing around 5,000 rib fractures from 660 computed tomography 153 (CT) scans. The dataset organizes detected rib fractures into 4 clinical categories (i.e., buckle, nondisplaced, displaced, and segmental rib fractures). As we use low-resolution images, we disregard segmental rib fractures and classify 3 types of rib fractures (i.e., buckle, nondisplaced, and displaced). For each annotated fracture area, we calculate its center and resize the center-cropped 64mm×64mm×64mm image into 28×28×28. The official split of training, validation and test se

In [ ]:
x, y = test_dataset[0]

print(x.shape, y.shape)

(1, 64, 64, 64) (1,)


In [ ]:
print(x[0][0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
os.listdir(test_dir)

['[0]', '[2]', '[1]']

In [ ]:
label_dir = os.path.join(test_dir, '[0]')

In [ ]:
print(label_dir)

./fracturemnist3d/test/[0]


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Set the directory containing the images
label_dir = os.path.join(test_dir, '[1]')

# Select the base name of the image you want to display, e.g., '1000'
base_image = '100'

# Create a list to store the image slices and filenames
image_slices = []
image_names = []

# Load the slices from 0 to 63
for i in range(64):
    # Construct the image file name
    img_name = f"{base_image}_{i}.png"
    img_path = os.path.join(label_dir, img_name)

    # Load the image and append it to the list
    if os.path.exists(img_path):
        image_slices.append(Image.open(img_path))
        image_names.append(img_name)
    else:
        print(f"Image {img_name} not found")

# Plot the slices
fig, axes = plt.subplots(8, 8, figsize=(12, 12))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < len(image_slices):
        ax.imshow(image_slices[i], cmap='gray')
        ax.set_title(image_names[i], fontsize=8)  # Display filename below each image
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Create a custom dataset class to handle 2D slices
class SliceDataset(data.Dataset):
    def __init__(self, dir, transform=None):
        self.dir = dir
        self.transform = transform
        self.labels = []
        self.filenames = []
        self.image_ids = []
        for label in os.listdir(dir):
            label_value = int(label.strip('[]'))
            label_dir = os.path.join(dir, label)
            for filename in os.listdir(label_dir):
                image_id = filename.split('_')[0]  # Extract 3D image ID (e.g., '12' from '12_0.png')
                self.labels.append(label_value)
                self.filenames.append(os.path.join(label_dir, filename))
                self.image_ids.append(image_id)  # Store the image ID for tracking

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        img = Image.open(filename)
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx], self.image_ids[idx]

# # Create data loaders for the 2D slices
# data_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[.5], std=[.5])
# ])
# preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    #torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[.5], std=[.5])
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[.5], std=[.5])
])

train_slice_dataset = SliceDataset(train_dir, transform=train_transform)
valid_slice_dataset = SliceDataset(valid_dir, transform=test_transform)
test_slice_dataset = SliceDataset(test_dir, transform=test_transform)

In [ ]:
# Custom DataLoader to ensure batches of 64 slices from the same 3D image
def collate_fn(batch):
    batch_images = []
    batch_labels = []
    current_image_id = batch[0][2]  # Get the 3D image ID of the first element in the batch

    # Print the 3D image ID being processed
    print(f"Processing 3D image ID: {current_image_id}")

    for img, label, image_id in batch:
        if image_id != current_image_id:
            raise ValueError(f"Mixed slices in batch. Expected image ID {current_image_id}, but got {image_id}")
        batch_images.append(img)
        batch_labels.append(label)

    return torch.stack(batch_images), torch.tensor(batch_labels) # Stack the images as tensors

In [ ]:
lr = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 10
train_loader = data.DataLoader(dataset=train_slice_dataset, batch_size=BATCH_SIZE, shuffle=True, )
valid_loader = data.DataLoader(dataset=valid_slice_dataset, batch_size=BATCH_SIZE, shuffle=True, )
test_loader = data.DataLoader(dataset=test_slice_dataset, batch_size=BATCH_SIZE, shuffle=False,  )

In [ ]:

# Check the data loader by fetching one batch
data_iter = iter(train_loader)
images, labels, slice_numbers = next(data_iter)
print(f"Batch size: {images.size()}, Labels: {labels}, Slice numbers: {slice_numbers}")

Batch size: torch.Size([64, 3, 224, 224]), Labels: tensor([0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 1, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0,
        2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 0, 1, 2, 0, 0, 2, 1, 0, 1, 0,
        1, 0, 2, 1, 0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 2, 1]), Slice numbers: ('44', '209', '246', '219', '990', '127', '273', '747', '781', '154', '939', '1001', '312', '775', '318', '566', '983', '990', '807', '51', '390', '893', '310', '124', '543', '740', '959', '962', '267', '132', '206', '377', '268', '382', '46', '822', '846', '871', '807', '144', '167', '645', '814', '556', '183', '700', '876', '999', '817', '489', '506', '244', '79', '880', '217', '228', '777', '327', '934', '204', '541', '382', '400', '646')


# MedVit Training

In [ ]:
# Install MedVit
!git clone https://github.com/Omid-Nejati/MedViT.git

In [ ]:
# Change directory to MedVit
%cd ./MedViT

/home/exouser/Documents/RRR_MedVit/MedViT


/home/exouser/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# Check Folder Content
%ls

Colab_MedViT.ipynb*  LICENSE*      fracturemnist3d/   utils.py*
CustomDataset/       MedViT.py*    images/
CustomDataset.md*    README.md*    requirements.txt*
Instructions.ipynb*  __pycache__/  synapsemnist3d/


In [ ]:
# Install Dependencies
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from MedViT import MedViT_small as tiny, MedViT_base as base
model = base()

initialize_weights...


In [ ]:
# Assuming you're using a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
model = model.to(device)

In [ ]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=3, bias=True)

In [ ]:
model = model.to(device)

In [ ]:
print(task)

multi-class


In [ ]:
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
# Train the model
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]')
    model.train()  # Set model to training mode

    for inputs, targets, slice_numbers in tqdm(train_loader):  # This should now work correctly
        inputs, targets = inputs.to(device), targets.to(device) # Move to GPU

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)

        # Compute loss
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)  # Ensure targets are float
            loss = criterion(outputs, targets)  # Binary cross-entropy loss
        else:
            targets = targets.squeeze().long()  # Ensure targets are long for class indices
            loss = criterion(outputs, targets)  # Cross-entropy loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class indices
        train_total += targets.size(0)  # Total number of targets
        train_correct += (predicted == targets).sum().item()  # Count correct predictions

    # Calculate and print training accuracy for the epoch
    train_accuracy = 100 * train_correct / train_total
    print(f'Training Accuracy: {train_accuracy:.2f}%')


Epoch [1/10]


100%|███████████████████████████████████████| 1027/1027 [08:51<00:00,  1.93it/s]


Training Accuracy: 49.38%
Epoch [2/10]


100%|███████████████████████████████████████| 1027/1027 [08:06<00:00,  2.11it/s]


Training Accuracy: 53.20%
Epoch [3/10]


...

Training Accuracy: 70.24%
Epoch [9/10]


100%|███████████████████████████████████████| 1027/1027 [08:14<00:00,  2.08it/s]


Training Accuracy: 72.22%
Epoch [10/10]


100%|███████████████████████████████████████| 1027/1027 [07:53<00:00,  2.17it/s]

Training Accuracy: 74.22%


In [ ]:
torch.save(model.state_dict(), 'medvit_fracturemninst3_base_weights.pth')

In [ ]:
# Save the entire model
torch.save(model, 'medvit_fracturemninst3_base_model.pth')

# MedVit Test

In [ ]:
model = torch.load('medvit_fracturemninst3d_base_model.pth')

/tmp/ipykernel_5722/2898604408.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('medvit_fracturemninst3d_base_model.pth')


In [ ]:

# Validation loop
model.eval()  # Set model to evaluation mode
if task == "multi-label, binary-class":
    all_labels = []
    all_preds = []
    total_correct = 0
    with torch.no_grad():
        for data in tqdm(test_loader):
            inputs, labels, slice_numbers = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Convert 1-channel to 3-channel
            # inputs = inputs.repeat(1, 3, 1, 1)

            outputs = model(inputs)
            outputs = torch.sigmoid(outputs)
            print("outputs", outputs) # Apply sigmoid for multi-label classification
            predicted = (outputs > 0.5).int()  # Thresholding for binary classification

            total_correct += (predicted == labels).sum().item()
            # Collect all labels and predictions for metric calculation
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(predicted.cpu().numpy().flatten())

    accuracy = total_correct / (len(test_loader.dataset) * n_classes) * 100
    print(f'Validation Accuracy: {accuracy:.2f}%')
    # Calculate Precision, Recall, and F1 Score using sklearn
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')

else:  # Multi-class classification
    total_correct = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data in tqdm(test_loader):
            inputs, labels, slice_numbers = data
            inputs, labels = inputs.to(device), labels.to(device)

             # Convert 1-channel to 3-channel
            # inputs = inputs.repeat(1, 3, 1, 1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get the predicted class

            total_correct += (predicted == labels).sum().item()
            # Collect all labels and predictions for metric calculation
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = total_correct / len(test_loader.dataset) * 100
    print(f'Test Accuracy: {accuracy:.2f}%')
    # Calculate Precision, Recall, and F1 Score using sklearn
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')

100%|█████████████████████████████████████████| 240/240 [00:46<00:00,  5.19it/s]


Test Accuracy: 45.31%
Precision: 0.42
Recall: 0.40
F1 Score: 0.40


#Shannon entropy voting: GPU optimized

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Function to calculate entropy for each slice's probability (on GPU)
def calculate_entropy_gpu(probabilities):
    # Entropy formula: H(X) = -p*log(p) - (1-p)*log(1-p)
    entropy_values = -probabilities * torch.log(probabilities + 1e-10) - (1 - probabilities) * torch.log(1 - probabilities + 1e-10)
    return entropy_values

# Function to normalize entropies (on GPU)
def normalize_entropies_gpu(entropies):
    total_entropy = torch.sum(entropies)
    if total_entropy == 0:
        return torch.ones_like(entropies) / len(entropies)  # Handle zero entropy case
    return entropies / total_entropy

# Function to combine probabilities using the normalized entropy as weights (on GPU)
def combine_probabilities_gpu(probabilities, weights):
    return torch.sum(probabilities * weights)

# Function to calculate 3D image-wise metrics for multiclass classification
def calculate_3d_image_metrics_gpu(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    true_labels = []
    predicted_labels = []

    for batch_images, batch_labels, slice_numbers in tqdm(test_loader):
        batch_images = batch_images.to(device)  # Move batch to GPU
        batch_labels = torch.tensor(batch_labels).to(device)  # Move labels to GPU

        # Step 1: Get prediction for each individual slice in the batch
        with torch.no_grad():
            outputs = model(batch_images)  # Model outputs for the 64 slices
            softmax_outputs = F.softmax(outputs, dim=1)  # Apply softmax to get class probabilities
            slice_probs, predicted_classes = torch.max(softmax_outputs, dim=1)  # Get max probability and predicted class for each slice

        # Step 2: Calculate entropies for the 64 slices (on GPU)
        entropies = calculate_entropy_gpu(slice_probs)

        # Step 3: Normalize the entropies (on GPU)
        weights = normalize_entropies_gpu(entropies)

        # Step 4: Combine the probabilities using the entropy-based weights (on GPU)
        combined_probs = combine_probabilities_gpu(slice_probs, weights)
        combined_class = torch.argmax(combined_probs)  # Get the predicted class for the combined result

        # Step 5: Get the true label (all slices in the batch have the same label)
        true_label = batch_labels[0].item()  # All slices have the same label
        predicted_labels.append(combined_class.item())  # Store predicted class
        true_labels.append(true_label)  # Store true class

    # Calculate metrics (on CPU for sklearn compatibility)
    accuracy = sum(np.array(true_labels) == np.array(predicted_labels)) / len(true_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')  # Use 'macro' for multiclass
    recall = recall_score(true_labels, predicted_labels, average='macro')  # Use 'macro' for multiclass
    f1 = f1_score(true_labels, predicted_labels, average='macro')  # Use 'macro' for multiclass
    #auc_roc = roc_auc_score(true_labels, predicted_labels, multi_class='ovr')  # Use 'ovr' for multiclass

    return accuracy, precision, recall, f1 # auc_roc

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the appropriate device

# Assuming test_loader is already defined
accuracy, precision, recall, f1 = calculate_3d_image_metrics_gpu(model, test_loader, device)

print(f"3D Image-wise Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
# print(f"A#UC-ROC: {auc_roc:.2f}")


  0%|                                                   | 0/240 [00:00<?, ?it/s]/tmp/ipykernel_5722/603199177.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_labels = torch.tensor(batch_labels).to(device)  # Move labels to GPU
100%|█████████████████████████████████████████| 240/240 [00:46<00:00,  5.17it/s]

3D Image-wise Accuracy: 37.50%
Precision: 0.12
Recall: 0.33
F1-score: 0.18



/home/exouser/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Function to calculate entropy for each slice's probability (on GPU)
def calculate_entropy_gpu(img):
    #Histogram based entropy
    pixel_values = img.flatten()
    hist, _ = np.histogram(pixel_values, bins=256, range=(0, 255), density=True)
    entropy_values = entropy(hist, base=2)
    return entropy_values

# Function to normalize entropies (on GPU)
def normalize_entropies_gpu(entropies):
    total_entropy = torch.sum(entropies)
    if total_entropy == 0:
        return torch.ones_like(entropies) / len(entropies)  # Handle zero entropy case
    return entropies / total_entropy

# Function to combine probabilities using the normalized entropy as weights (on GPU)
def combine_probabilities_gpu(probabilities, weights):
    return torch.sum(probabilities * weights)

# Function to calculate 3D image-wise metrics
def calculate_3d_image_metrics_gpu(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    true_labels = []
    predicted_labels = []

    for batch_images, batch_labels in test_loader:
        batch_images = batch_images.to(device)  # Move batch to GPU
        batch_labels = torch.tensor(batch_labels).to(device)  # Move labels to GPU

        # Step 1: Get prediction for each individual slice in the batch
        with torch.no_grad():
            outputs = model(batch_images)  # Model outputs for the 64 slices
            softmax_outputs = F.softmax(outputs, dim=1)  # Apply softmax to get probabilities
            slice_probs = softmax_outputs[:, 1]  # Get probability for class 1 (binary classification)

        # Step 2: Calculate entropies for the 64 slices (on GPU)
        entropies = calculate_entropy_gpu(batch_images)

        # Step 3: Normalize the entropies (on GPU)
        weights = normalize_entropies_gpu(entropies)

        # Step 4: Combine the probabilities using the entropy-based weights (on GPU)
        combined_probability = combine_probabilities_gpu(slice_probs, weights)

        # Step 5: Get the true label (all slices in the batch have the same label) and make a prediction
        true_label = batch_labels[0]  # All slices have the same label
        predicted_label = np.argmax(combined_probability)

        # Store true and predicted labels for final metrics calculation (move to CPU for sklearn)
        true_labels.append(true_label.item())
        predicted_labels.append(predicted_label)

    # Calculate accuracy, precision, recall, and F1-score (on CPU for sklearn compatibility)
    accuracy = sum(np.array(true_labels) == np.array(predicted_labels)) / len(true_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    auc_roc = roc_auc_score(true_labels, predicted_labels)

    return accuracy, precision, recall, f1, auc_roc

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to the appropriate device

# Assuming test_loader is already defined
accuracy, precision, recall, f1, auc_roc = calculate_3d_image_metrics_gpu(model, test_loader, device)

print(f"3D Image-wise Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"AUC-ROC: {auc_roc:.2f}")

3D Image-wise Accuracy: 43.20%
Precision: 0.39
Recall: 0.41
F1-score: 0.36
AUC-ROC: 0.30
